In [ ]:
# TODO 資料整理，各球隊作為主場球隊時 進球表現

In [1]:
# TODO 構造特徵

# 主場球隊名
# 客場球隊名
# 全場 主場球隊進球數
# 全場 客場球隊進球數
# 比賽結果 ( H= 主場贏, D= 平局, A= 客場贏)
# 比賽結果 ( 比分 )
# 主場淨勝球數
# 客場的淨胜球數
# 主場累計得分
# 客場累計得分
# Poisson各比分機率

# 統計某支隊伍最近三場比賽的表現
# HM(n) 代表主場球隊上一次比賽的輸贏，
# AM(n) 代表客場球隊上一次比賽是輸贏。

# 加入比賽輪次為特徵（第幾個輪次MW）

In [ ]:
# TODO 清洗資料
# 每場賽事特徵值皆是依據過往比賽所統計出來，因此最一開始前幾比資料無法統計信息不足，必須拋棄。

In [ ]:
# TODO 分析敘述資料、解決樣本不均衡(暫定)
# 依照先前統計發現主場獲勝的比例接近 50% ，對於三分類(主、客、和)，標籤比例是不均衡的。
# 因此需簡化為二分類問題，也就是主場球隊會不會勝利。

In [ ]:
# TODO 將數據拆分為「特徵值」與「標籤值」

In [ ]:
# TODO 數據標準化
# 主場淨勝球數、客場的淨胜球數、主場累計得分、客場累計得分 數據內容區間範圍太大因此需正規劃

In [ ]:
# TODO 轉換特徵數據型態、預覽

In [ ]:
# TODO 數據關聯圖形化描述
# 皮爾遜相關熱力圖

In [ ]:
# TODO 再次清潔數據
# 依照關聯圖移除無異議數據

In [2]:
# TODO 切割數據
# 將數據集隨機分成為訓練集和測試集

In [ ]:
# TODO 建立模型
# 支持向量機介紹(SVM)

In [ ]:
# TODO 結果評估

In [ ]:
# TODO 參數調整

In [ ]:
# TODO 結論